In [2]:
import re
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql.cursors
from datetime import datetime

# Global variables
mysql_host = '172.21.1.151'
mysql_port = 3306
mysql_user = 'xli'
mysql_password = '123456'
mysql_db = 'kdwtemp2'
mysql_charset = 'utf8mb4'

def main():
    #确保端口设置为HS2服务，而不是Beeswax服务。在Cloudera的管理集群中，HS2的默认端口是21050
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()
    #cur.execute('SHOW TABLES')
    # print cur.fetchall()
    cur.close()
    conn.close()
def easy_print(tname, records=10):
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT * FROM ' + tname + ' LIMIT 10' 
    cur.execute(sql)
    read_in = as_pandas(cur)    
    cur.close()
    conn.close()
    
    #print in a format
    columns = read_in.columns.values
    values = read_in.values
    records = []
    for v in values:
        records.append(zip(columns, v))
    
    for ix,r in enumerate(records):
        print('----------第%s条记录--------' % (ix+1))
        for k, v in r:
            print('%s: %s' % (k, v))

def connect_Mysql(sql_columns_select,tname): #连接Mysql数据库
    mysqlConnection = pymysql.connect(host=mysql_host,
                                      port=mysql_port,
                                      user=mysql_user,
                                      password=mysql_password,
                                      db=mysql_db,
                                      charset=mysql_charset)
                                      #cursorclass=pymysql.cursors.DictCursor)
    try:
        sql = 'SELECT '+ sql_columns_select + ' FROM '+ tname #+ ' LIMIT 100'
        df = pd.read_sql(sql,mysqlConnection)
        return df
    except:
        mysqlConnection.close()            

def song_release_time(): #抽取出songid对应的歌曲发行时间
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()
    sql = """
    SELECT songid,releasetime FROM kdw_tbl_song_info"""
    cur.execute(sql)
    df = as_pandas(cur)
    """# 清洗掉不符合时间格式的数据行
    pattern1 = re.compile(r'\d\d\d\d-\d\d-\d\d)
    df = df[map(lambda x: re.match(pattern1,x)!=None,df['releasetime'])]
    #提取出年份
    df['releasetime']=pd.to_datetime(df['releasetime'], format='%Y-%m-%d')
    df['year'] = [i.year for i in df['releasetime']]  
    #df.to_csv("E:/raw_data.csv")"""    
    cur.close()
    conn.close()  
    return df

def year_stage(data): #给年份分段
    if data >= 2000:
        return 5
    elif data >= 1990 and data < 2000:
        return 4
    elif data >= 1980 and data < 1990:
        return 3
    elif data >= 1970 and data < 1980:
        return 2
    else:
        return 1

def age_srt(df): #通过歌曲release time判断发行时间年龄段
    df['age_year'] = [year_stage(i) for i in df['year']]
    return df   

def read_play_cnt(): #抽取出用户点歌记录??并累加点歌次数
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()
    sql = """
    SELECT customerid,songid,playcount as song_play_cnt FROM kdw_tbl_song_menu_event_statics"""
    cur.execute(sql)
    df = as_pandas(cur)
    #在思考要不要这里groupby
    #后续需要与用户表做表连
    cur.close()
    conn.close()    
    return df    

def age_play_record(df1,df2):
    # ？？如何从歌曲发行信息查到songid对应的年龄tag是一个问题
    df = pd.merge(df1,df2,on='songid')
    return df

    
def age_song_language():
    df = connect_Mysql('SongID,LanguageType','kdw_tbl_song')
    # 待思考如何从语言给权重 

if __name__ == '__main__':
    df_age_year = song_release_time()

In [6]:
list(df_age_year['releasetime']).index('1999-01-28;')

19815